In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [2]:
#check if eager mode is working
tfe.executing_eagerly()

True

In [3]:
#get version of tf in ypur computer
tf.__version__

'1.13.1'

In [4]:
#test the eager execution
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

hello, [[4.]]


In [5]:
import pennylane as qml
from pennylane import numpy as np

In [6]:
#define circuit and qnode
dev2 = qml.device('default.qubit', wires=3)

@qml.qnode(dev2, interface='tfe')
def circuit2(p1, p2):
    qml.Rot(p1[0], p1[1], p1[2], wires=1)
    qml.Rot(p2[0], p2[1], p2[2], wires=2)
    return qml.expval.PauliZ(0), qml.expval.PauliZ(1), qml.expval.PauliZ(2)

In [24]:
t1 = tfe.Variable(tf.random.normal([3], dtype=tf.float64))
t2 = tfe.Variable(tf.random.normal([3], dtype=tf.float64))
#t1 = tfe.Variable([0, np.pi, 0], dtype=tf.float64)
#t2 = tfe.Variable([0, np.pi, 0], dtype=tf.float64)

In [25]:
t1

<tf.Variable 'Variable:0' shape=(3,) dtype=float64, numpy=array([-0.24696728,  0.76951318, -0.26863317])>

In [26]:
opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)

In [27]:
#using J=[1,-1] as default couplings
def cost(var):
#   J= tf.constant([1, -1])
    spins = circuit2(var[0], var[1])
    energy = -(1*spins[0]*spins[1])-(-1*spins[1]*spins[2])
    return energy

In [28]:
steps = 100

for i in range(steps):
    with tf.GradientTape() as tape:
        loss = cost([t1, t2])
        grads = tape.gradient(loss, [t1, t2])

    opt.apply_gradients(zip(grads, [t1, t2]), global_step=tf.train.get_or_create_global_step())
    
    if i % 5 == 0:
        print("Step {}: t1 = [{:f}, {:f}, {:f}], t2 = [{:f}, {:f}, {:f}], loss = {}".format(i,
                *t1.numpy(), *t2.numpy(), loss.numpy()))

Step 0: t1 = [-0.246967, 0.756233, -0.268633], t2 = [-1.401288, 0.670313, 0.611821], loss = -0.13708429784352494
Step 5: t1 = [-0.246967, 0.663433, -0.268633], t2 = [-1.401288, 0.930258, 0.611821], loss = -0.275460537793314
Step 10: t1 = [-0.246967, 0.516825, -0.268633], t2 = [-1.401288, 1.287442, 0.611821], loss = -0.5496015696296994
Step 15: t1 = [-0.246967, 0.329050, -0.268633], t2 = [-1.401288, 1.732467, 0.611821], loss = -0.9971901565841157
Step 20: t1 = [-0.246967, 0.161894, -0.268633], t2 = [-1.401288, 2.182695, 0.611821], loss = -1.4758278042230473
Step 25: t1 = [-0.246967, 0.064369, -0.268633], t2 = [-1.401288, 2.538981, 0.611821], loss = -1.7820146837012802
Step 30: t1 = [-0.246967, 0.022786, -0.268633], t2 = [-1.401288, 2.777232, 0.611821], loss = -1.9188822480212904
Step 35: t1 = [-0.246967, 0.007681, -0.268633], t2 = [-1.401288, 2.924604, 0.611821], loss = -1.9710478454170068
Step 40: t1 = [-0.246967, 0.002543, -0.268633], t2 = [-1.401288, 3.013078, 0.611821], loss = -1.98